In [1]:
import json
import pandas as pd
import glob

In [2]:
data_path = './models/'

In [3]:
embedding_models_en = [
  'ccnews_en_embedding',
  'wikipedia_en_embedding',
  'ud_en_embedding',
  'combined_en_embedding',
]

embedding_models_cs = [
  'sumeczech_cs_embedding',
  'wikipedia_cs_embedding',
  'ud_cs_embedding',
  'combined_cs_embedding',
]

embedding_models_sk = [
  'slovaksum_sk_embedding',
  'squad_sk_embedding',
  'wikipedia_sk_embedding',
  'ud_sk_embedding',
  'combined_sk_embedding',
]

embedding_model_cs_sk = [
  'ud_cs_sk_embedding',
  'wikipedia_cs_sk_embedding'
]

In [4]:
downstream_en = [
  'fever',
  'liar',
  'multifc',
  'xfact',
  'combined_claim_en',
]

downstream_cs = [
  'csfever',
  'ctkfacts',
  'demagog',
  'combined_claim_cs',
]

downstream_sk = [
  'demagog',
  'combined_claim_sk',
]

In [5]:
current_downstream = downstream_sk
current_embedding = embedding_model_cs_sk

In [6]:
results = dict()

for embed in current_embedding:
    for task in current_downstream:
      print(data_path + embed + '/' + task + '/LSTMClassifier/metrics*.json')
      metrcis_file = glob.glob(data_path + embed + '/' + task + '/LSTMClassifier/metrics*.json')
      metrcis_file.sort()
      # load all metrics file
      metrics = []
      for file in metrcis_file:
        with open(file) as json_file:
          metrics.append(json.load(json_file))

      # add data to dataframe
      df = pd.DataFrame()
      for idx, metric in enumerate(metrics):
        # get seed idx
        seed = metrcis_file[idx].split('/')[-1].split('.')[0].split('-')[-1]
        record_df = pd.DataFrame.from_dict(metric, orient='index').T
        record_df['seed'] = seed
        df = pd.concat([df, record_df], ignore_index=True)
      
      results[embed + '_' + task] = df

./models/ud_cs_sk_embedding/demagog/LSTMClassifier/metrics*.json
./models/ud_cs_sk_embedding/combined_claim_sk/LSTMClassifier/metrics*.json
./models/wikipedia_cs_sk_embedding/demagog/LSTMClassifier/metrics*.json
./models/wikipedia_cs_sk_embedding/combined_claim_sk/LSTMClassifier/metrics*.json


In [7]:
results

{'ud_cs_sk_embedding_demagog':     test_f1  test_accuracy  test_precision  test_recall seed
 0  0.716074       0.600207        0.701584     0.731418  111
 1  0.718507       0.600287        0.709125     0.729253  123
 2  0.761112       0.644770        0.712571     0.817647  321
 3  0.718058       0.606824        0.725056     0.712597   42
 4  0.719764       0.601562        0.719826     0.720638  456
 5  0.746165       0.630660        0.736622     0.756844  654
 6  0.713425       0.597975        0.698411     0.730220  789
 7  0.716933       0.598294        0.712392     0.721941  888
 8  0.704421       0.591916        0.741363     0.671869  987
 9  0.732559       0.617188        0.739495     0.727322  999,
 'ud_cs_sk_embedding_combined_claim_sk':     test_f1  test_accuracy  test_precision  test_recall seed
 0  0.737045       0.636813        0.735299     0.740454  111
 1  0.697784       0.591903        0.697114     0.698924  123
 2  0.738180       0.631236        0.723793     0.753653  321

In [8]:
datatable_f1 = pd.DataFrame(columns=current_embedding, index=current_downstream)
datatable_accuracy = pd.DataFrame(columns=current_embedding, index=current_downstream)


for embed in current_embedding:
    for task in current_downstream:
        data = results[embed + '_' + task]

        # get mean and std of test_f1 and test_accuracy not base on the seed
        mean_test_f1 = data['test_f1'].mean()
        std_test_f1 = data['test_f1'].std()
        mean_test_accuracy = data['test_accuracy'].mean()
        std_test_accuracy = data['test_accuracy'].std()

        datatable_f1.loc[task, embed] = str(round(mean_test_f1*100, 2)) + ' ± ' + str(round(std_test_f1*100, 2))
        datatable_accuracy.loc[task, embed] = str(round(mean_test_accuracy*100, 2)) + ' ± ' + str(round(std_test_accuracy*100, 2))

In [9]:
print(datatable_f1.to_latex().replace(' ±', '\% $\pm$'))

\begin{tabular}{lll}
\toprule
 & ud_cs_sk_embedding & wikipedia_cs_sk_embedding \\
\midrule
demagog & 72.47\% $\pm$ 1.71 & 71.99\% $\pm$ 2.22 \\
combined_claim_sk & 71.96\% $\pm$ 1.78 & 72.3\% $\pm$ 1.64 \\
\bottomrule
\end{tabular}



In [33]:
results['wikipedia_cs_sk_embedding_demagog'].iloc[[0, 5]].describe()

,test_f1,test_accuracy,test_precision,test_recall
count,2.000000,2.000000,2.000000,2.000000
mean,0.749655,0.633131,0.720531,0.782778
std,0.007967,0.012627,0.027377,0.015734
min,0.744022,0.624203,0.701172,0.771652
25%,0.746838,0.628667,0.710851,0.777215
50%,0.749655,0.633131,0.720531,0.782778
75%,0.752472,0.637596,0.730210,0.788341
max,0.755288,0.642060,0.739889,0.793904
